# 학습 목표

- 레이어의 개념을 이해한다.
- 딥러닝 모델 속 각 레이어(Embedding, RNN, LSTM)의 동작 방식을 이해한다.
- 데이터의 특성을 고려한 레이어를 설계하고, 이를 Tensorflow로 정의하는 법을 배운다.

## 분포 가설과 분산 표현

### 단어의 분산 표현(Distributed Representation)

- Sparse Representation은 벡터 차원에 의미를 둬서 그에 맞게 단어들을 매핑하는 것
- 그러나 Distributed Representation은 그렇게 의미를 부여하지 않음
- 대신 문맥에서 나타나는 위치를 토대로 `'유사한 맥락에서 동일한 위치에 나오면 의미가 비슷할 것이다'`라는 분포 가설(distribution hypothesis)를 따라서 단어들의 벡터를 만듦
- 이 때 벡터의 차원은 고정시킴
- 그래서 유사한 문맥에서 동일한 위치에 나오는 단어들은 벡터로 표현했을 때 서로 거리를 가깝게 해주는 게 Distributed Representation의 핵심
- 그리고 벡터의 차원들은 각각 특정 의미를 내포한다라기 보다 그냥 차원 전체(벡터 전체)에 있어서 그 의미가 고루 퍼져 있다라고 여길 수 있음

> #### 이렇게 그냥 n개 단어로, k차원을 가진 벡터들을 만드는 걸 `Embedding`레이어가 한다.
> #### 그리고 우린 그냥 n과 k만 정해주면 된다.
> #### 그리고 여기서 `Embedding` 레이어는 그 벡터들(단어 사전)을 만드는 게 일인데, 그 단어 사전 자체가 Weight, 즉 파라미터이다.

## 단어를 부탁해! Embedding 레이어

- 단어사전의 사이즈를 우리가 정해준다.
- 그리고 이 사이즈가 곧 Weight의 크기이다.


#### Lookup table
- 저 weight이 담긴 단어 사전(즉, n개의 단어 x k개의 차원)은 추후 알고 싶은 단어의 벡터를 뽑아온다는 의미에서 Lookup table로도 불린단다.
- 그리고 이렇게 단어사전에 word -> vector로 만드는 과정이 바로 핵심!
- 그리고 그 핵심 원리의 기본은 one-hot encoding!

#### one-hot encoding

- 단점이 있는데,
- 내적의 값이 0인 벡터들(직교하는 벡터들)은 1과 1이 만나는 지점이 없는 서로 독립된 단어가 된다.(그런데 이들끼리 연관이 있으면...?)
- 차원의 저주도 있다.
  - 단어 수만큼 차원이 늘어나면 골치 아프다..

#### 실험! - one-hot encoding을 linear layer로 적용해보기
재밌는 상상을 한 번 해봅시다. 선형변환 담당 Linear 레이어를 잊지 않았죠? 원-핫 인코딩에 Linear 레이어를 적용하면 어떻게 될까요? 단 하나의 인덱스만 1이고 나머지가 모두 0인 극단적인 벡터지만 어쨌건 고차원 벡터이니 적용해 볼 수 있잖아요! 백문이 불여일견, 소스로 한 번 확인해 보죠, 우선 원-핫 벡터를 먼저 생성합니다!

In [1]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy())    # 원-핫 인코딩 벡터를 출력해 봅시다.

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


In [2]:
one_hot

<tf.Tensor: shape=(10, 8), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)>

> tf.one_hot()은 정수로 된 어레이를 받아서 해당 정수에 맞게 one-hot encoding된 행렬을 내뱉어 준다.

어때요, 원-핫 인코딩의 결과가 여러분이 상상한 것과 동일한가요? 이제 생성된 원-핫 벡터를 Linear 레이어에 넣어보죠. 놀라운 결과가 기다리고 있답니다..!

In [3]:
distribution_size = 2   # 보기 좋게 2차원으로 분산 표현하도록 하죠!
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

print("Linear Weight")
print(linear.weights[0].numpy())

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight
[[-0.05541933  0.7289257 ]
 [ 0.5526576   0.17332882]
 [-0.4281848   0.04122239]
 [-0.6178259   0.38403618]
 [ 0.6672548  -0.6742032 ]
 [-0.05171949 -0.08872664]
 [ 0.7648511  -0.4172705 ]
 [ 0.06281644  0.6062473 ]]

One-Hot Linear Result
[[-0.05541933  0.7289257 ]
 [-0.05541933  0.7289257 ]
 [-0.05541933  0.7289257 ]
 [-0.05541933  0.7289257 ]
 [ 0.5526576   0.17332882]
 [-0.4281848   0.04122239]
 [-0.6178259   0.38403618]
 [ 0.6672548  -0.6742032 ]
 [ 0.6672548  -0.6742032 ]
 [ 0.6672548  -0.6742032 ]]


> #### note:
> - 위의 인풋 데이터와 Dense layer를 지난 아웃풋을 비교해보면
> - 1이 있는 자리(즉, 정수 인코딩에서 1이 있는 자리)만 가중치와 곱해져서 나온 것을 알 수 있다.
> - 이런 식으로 문장을 정수 인코딩(단어 사전의 인덱스)해주고 모델에 넣으면 사실 `Embedding` 레이어는 정수 인코딩을 one-hot encoding으로 바꾸고 선형변환을 통해 의미를 가진 벡터를 만들어낸단다.

In [6]:
linear.weights

[<tf.Variable 'dense/kernel:0' shape=(8, 2) dtype=float32, numpy=
 array([[-0.05541933,  0.7289257 ],
        [ 0.5526576 ,  0.17332882],
        [-0.4281848 ,  0.04122239],
        [-0.6178259 ,  0.38403618],
        [ 0.6672548 , -0.6742032 ],
        [-0.05171949, -0.08872664],
        [ 0.7648511 , -0.4172705 ],
        [ 0.06281644,  0.6062473 ]], dtype=float32)>]

In [7]:
one_hot_linear

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[-0.05541933,  0.7289257 ],
       [-0.05541933,  0.7289257 ],
       [-0.05541933,  0.7289257 ],
       [-0.05541933,  0.7289257 ],
       [ 0.5526576 ,  0.17332882],
       [-0.4281848 ,  0.04122239],
       [-0.6178259 ,  0.38403618],
       [ 0.6672548 , -0.6742032 ],
       [ 0.6672548 , -0.6742032 ],
       [ 0.6672548 , -0.6742032 ]], dtype=float32)>

#### tensorflow에서 임베딩 레이어 선언하는 법

In [8]:
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장입니다.

print("Embedding을 진행할 문장:", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언할 것이고,
# 각 단어는 100차원으로 분산 표현 할 것입니다.

print("Embedding된 문장:", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

Embedding을 진행할 문장: (1, 3)
Embedding된 문장: (1, 3, 100)
Embedding Layer의 Weight 형태: (64, 100)


In [9]:
some_words

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[ 3, 57, 35]], dtype=int32)>

#### 임베딩 레이어의 주의사항
> #### note:
> - 해당 레이어는 그냥 단어를 대응해준 레이어에 불과해서 미분이 안된단다.
> - 따라서 신경망에 붙일 때 `어떤 연산 결과를 임베딩 레이어에 붙이는 것`은 불가능하다.
> - 그래서 `입력에 직접 연결해서 쓴다.`

## 순차적인 데이터! Recurrent 레이어 (1) RNN

- RNN의 입력으로 들어가는 `모든 단어만큼 Weight를 만드는 게 아님에 유의`
- `(입력의 차원, 출력의 차원)`에 해당하는 단 하나의 Weight를 순차적으로 업데이트하는 것이 RNN

#### 코드로 구현하는 RNN

In [10]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장:", sentence)

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위해 단어 매핑:", sentence_tensor.numpy())
print("입력 문장 데이터 형태:", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과:", emb_out.shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output):", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태:", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output):", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태:", rnn_fin_layer.weights[0].shape)

RNN에 입력할 문장: What time is it ?
Embedding을 위해 단어 매핑: [[2 3 0 1 4]]
입력 문장 데이터 형태: (1, 5)

Embedding 결과: (1, 5, 100)
Embedding Layer의 Weight 형태: (5, 100)

RNN 결과 (모든 Step Output): (1, 5, 64)
RNN Layer의 Weight 형태: (100, 64)

RNN 결과 (최종 Step Output): (1, 64)
RNN Layer의 Weight 형태: (100, 64)


In [12]:
sentence_tensor

<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[2, 3, 0, 1, 4]], dtype=int32)>

In [13]:
rnn_seq_out

<tf.Tensor: shape=(1, 5, 64), dtype=float32, numpy=
array([[[ 3.41891013e-02, -2.78892722e-02, -3.07099316e-02,
         -3.12659726e-03,  5.28826229e-02, -1.27824536e-02,
         -1.09696081e-02,  4.05459702e-02, -3.42771746e-02,
         -6.42082328e-03, -6.06246106e-02, -1.71578042e-02,
         -3.97295617e-02, -3.13773449e-03, -2.22241431e-02,
         -4.83743623e-02,  2.03773677e-02,  4.20067497e-02,
         -1.15800295e-02,  1.25815263e-02,  6.79355767e-03,
          3.30978297e-02, -3.94702666e-02,  3.20414379e-02,
          2.69302558e-02,  4.19907235e-02,  6.41035568e-03,
         -3.24814431e-02,  5.22524863e-02, -1.66594516e-02,
          4.97148372e-03, -6.90012844e-03,  2.34572720e-02,
          4.23527025e-02,  6.31098822e-03,  1.52858943e-02,
          9.05076601e-03,  4.98658456e-02, -3.10697220e-02,
         -1.17295736e-03, -9.48296674e-03, -5.86851835e-02,
          3.02995816e-02,  1.87483709e-02, -2.86785164e-03,
         -3.67441075e-03,  2.16524452e-02,  8.35

#### 위의 코드를 lstm으로 구현해보기(동일함)

In [14]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output):", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output): (1, 5, 64)
LSTM Layer의 Weight 형태: (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)


> #### 아이펠 설명
> 처음 보는 LSTM이라는 레이어가 등장했습니다!   
> Embedding 벡터의 차원수(unit)의 크기가 동일할 경우`(위 예에서는 units=64)`, Weight의 크기가 위에서 사용했던 SimpleRNN의 4배나 되는 것을 볼 수 있는데, 왜 이런 RNN 레이어가 등장하게 된 것일까요?

## 순차적인 데이터! Recurrent 레이어 (2) LSTM

#### 아이펠 설명 

LSTM 은 기본적인 바닐라(Simple) RNN보다 4배나 큰 Weight를 가지고 있음을 이전 스텝에서 확인했죠? 4배 깊은 RNN이라고 표현하기보다, 4종류의 서로 다른 Weight를 가진 RNN이라고 이해하시는 것이 좋습니다. 각 Weight들은 Gate라는 구조에 포함되어 어떤 정보를 기억하고, 어떤 정보를 다음 스텝에 전달할지 등을 결정합니다!

LSTM에는 Cell state 라는 새로운 개념이 추가되는데, 긴 문장이 들어와도 이 Cell state 를 통해 오래된 기억 또한 큰 손실 없이 저장해줍니다. 위의 두 번째 참고 링크를 보시면 수식으로도 이해하실 수 있을 것입니다. 그리고 앞서 언급한 Gate 들이 Cell state에 정보를 추가하거나 빼주는 역할을 합니다.

## GRU

#### 아이펠 설명 


LSTM은 GRU에 비해 Weight가 많기 때문에 충분한 데이터가 있는 상황에 적합하고, 반대로 GRU는 적은 데이터에도 웬만한 학습 성능을 보여주죠. 이것저것 사용해보며 여러분의 프로젝트에 적합한 레이어를 찾아보세요!

### 양방향(Bidirectional) RNN

마지막으로 살펴볼 것은 진행 방향에 변화를 준 RNN입니다. `날이 너무 [ ] 에어컨을 켰다` 라는 예문이 있다면, 빈칸에 들어갈 말이 '더워서'인 것은 어렵지 않게 유추할 수 있습니다. 그 근거는 아마도 `'에어컨을 켰다'` 는 말 때문이겠죠? 하지만 우리가 지금까지 배운 RNN은 모두 순방향이었기 때문에 에어컨이라는 정보가 없는 채로 빈칸에 들어갈 단어를 생성해야 합니다. 자칫하면 날이 너무 추워서 에어컨을 켰다 는 이상한 문장이 생성될지도 몰라요!

이를 해결하기 위해 제안된 것이 양방향(Bidirectional) RNN입니다. 말이 조금 어렵지만, 그저 `진행 방향이 반대`인 `RNN을 2개` 겹쳐놓은 형태랍니다!

원리만큼이나 간단하게 Tensorflow에서도 LSTM 등 모든 RNN 계열 레이어에 쉽게 적용시킬 수 있습니다. 사용하고자 하는 레이어를 `tf.keras.layers.Bidirectional()` 로 감싸주기만 하면 돼요!

양방향(Bidirectional) RNN이 필요한 상황은 어떤 것일까요? 문장 분석이나 생성보다는 주로 기계번역 같은 테스크에 유리합니다. 사람도 대화를 하면서 듣고 이해하는 것은 순차적으로 들으면서 충분히 예측을 동원해서 잘 해냅니다. 그러나 문장을 번역하려면 일단은 번역해야 할 문장 전체를 끝까지 분석한 후 번역을 시도하는 것이 훨씬 유리합니다. 그래서 자연어처리를 계속하면서 알게 되겠지만, 번역기를 만들 때 양방향(Bidirectional) RNN 계열의 네트워크, 혹은 동일한 효과를 내는 Transformer 네트워크를 주로 사용하게 될 것입니다.

In [20]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (최종 Step Output):", bi_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (최종 Step Output): (1, 5, 128)


Bidirectional RNN은 순방향 Weight와 역방향 Weight를 각각 정의하므로 우리가 앞에서 배운 RNN의 2배 크기 Weight가 정의됩니다. units를 64로 정의해 줬고, 입력은 Embedding을 포함하여 (1, 5, 100), 그리고 양방향에 대한 Weight를 거쳐 나올 테니 출력은 (1, 5, 128) 이 맞죠?